This notebook is a brief attempt to perform Logistic Regression using [Maximum Likelihood Estimation](https://neos-guide.org/content/logit) to estimate model coefficients, using a simple dataset. The emphasis here is not really on the modeling particulars, but to explore using numerical methods to estimate the parameters of a Logistic Regression model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

We'll use the well-known [Iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) for predicting Iris types from petal and sepal dimensions. For this example, we'll be using `petal_length` and `petal_width` to predict whether the Iris is of type *Iris-Virginica* or not.

In [3]:
iris = datasets.load_iris()
X = iris['data'][:, 2:]
y = (iris['target']==2).astype(np.int)

Let's see what happens with scikit-learn first, note that `C` is set to a large number to mimic Logisitc Regression with almost no regularization:

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [5]:
lr = LogisticRegression(class_weight='balanced', C=1e6)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=42)

In [7]:
lr.fit(X_train, y_train)

LogisticRegression(C=1000000.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [8]:
lr.intercept_, lr.coef_

(array([-56.4120616]), array([[8.21998445, 9.79924428]]))

In [9]:
y_pred_sklearn = lr.predict(X_test)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

In [11]:
print(classification_report(y_test, y_pred_sklearn))

             precision    recall  f1-score   support

          0       0.97      1.00      0.98        30
          1       1.00      0.93      0.97        15

avg / total       0.98      0.98      0.98        45



In [12]:
confusion_matrix(y_test, y_pred_sklearn)

array([[30,  0],
       [ 1, 14]])

Now let's try to fit a Logistic Regression model from scratch. First, we'll need to append a column of ones to our feature matrices to represent the intercept column.

In [13]:
X_train2, X_test2 = np.concatenate([np.ones([len(X_train), 1]), X_train], axis=1), np.concatenate([np.ones([len(X_test), 1]), X_test], axis=1)
X_train2, X_test2

(array([[1. , 1.6, 0.2],
        [1. , 1.5, 0.3],
        [1. , 1.4, 0.2],
        [1. , 5.9, 2.3],
        [1. , 1.6, 0.2],
        [1. , 5.2, 2.3],
        [1. , 4.5, 1.3],
        [1. , 5. , 1.7],
        [1. , 1.4, 0.2],
        [1. , 5.9, 2.1],
        [1. , 5.1, 2. ],
        [1. , 1.4, 0.2],
        [1. , 3.8, 1.1],
        [1. , 4.2, 1.5],
        [1. , 3.7, 1. ],
        [1. , 6. , 1.8],
        [1. , 1. , 0.2],
        [1. , 1.6, 0.2],
        [1. , 5.7, 2.5],
        [1. , 4.5, 1.5],
        [1. , 3.9, 1.1],
        [1. , 4.9, 1.8],
        [1. , 6.6, 2.1],
        [1. , 6.7, 2. ],
        [1. , 1.3, 0.4],
        [1. , 1.4, 0.2],
        [1. , 3.3, 1. ],
        [1. , 4.1, 1.3],
        [1. , 4.4, 1.4],
        [1. , 1.4, 0.2],
        [1. , 1.5, 0.4],
        [1. , 1.9, 0.4],
        [1. , 4.5, 1.5],
        [1. , 4. , 1.3],
        [1. , 1.7, 0.3],
        [1. , 1.4, 0.3],
        [1. , 4.7, 1.6],
        [1. , 5. , 1.9],
        [1. , 6.4, 2. ],
        [1. , 1.5, 0.4],


Now we will try to find the set of Logistic Regression coefficients maximizing the Log Likelihood for Logistic Regression ([see bottom Page 2 and top Page 3 for its gradient](https://web.stanford.edu/class/archive/cs/cs109/cs109.1178/lectureHandouts/220-logistic-regression.pdf)). We will specifically use gradient ascent (not descent, because we are maximizing) to iteratively determine the coefficients (`betas`), with a starting guess of all three coefficients (intercept plus two feature weights) being zero. Note that this will take many iterations.

In [14]:
betas = np.zeros(3)
err = 9999
learn = 1e-3

while err > 1e-6:
    step = np.dot(X_train2.T, y_train - (1/(1 + np.exp(-np.dot(X_train2, betas)))))
    betas_new = betas + learn * step
    err = np.linalg.norm(betas_new - betas)
    betas = betas_new

In [15]:
print(betas)
print(lr.intercept_, lr.coef_)

[-54.3444985    7.2544039   10.93128676]
[-56.4120616] [[8.21998445 9.79924428]]


We can see that the coefficients we've estimated by maximizing the Log Likelikhood function using gradient ascent are a bit off from what scikit-learn. This result could be due to there not being completely zero regularization in the model `lr` that we found using scikit-learn or the highly optimized algorithms scikit-learn uses compared to our barebones implementation which besides being slow, also hasn't really optimized for learning rate either.

Let's see how good our predictions compare now (note that we are using the logistic (sigmoid) function and mapping the resulting probability of being a "positive" -- the *Iris-Virginica* type to the class prediction):

In [16]:
y_pred = [int(x >= .5) for x in (1/(1 + np.exp(-np.dot(X_test2, betas))))]

In [17]:
sum(y_pred==y_pred_sklearn) / len(y_pred)

0.9777777777777777

In [18]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.94      1.00      0.97        30
          1       1.00      0.87      0.93        15

avg / total       0.96      0.96      0.95        45



In [19]:
confusion_matrix(y_test, y_pred)

array([[30,  0],
       [ 2, 13]])

Looks as if we are off by one false negative.